In [9]:
from agents.collection_agent import CollectionAgent
import os
from glob import glob
import gradio as gr
import pandas as pd
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [188]:
!pip install pymupdf4llm -q

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:

files = glob("/Users/admin/Projects/MathTestCreator/Documents/*")

In [4]:
import pymupdf
from agents.source_material import SourceMaterial

In [5]:
material = SourceMaterial(files[0])

/Users/admin/anaconda3/envs/testg/lib/python3.11/site-packages/langchain_community/document_loaders/parsers/pdf.py:322: UserWarning: Warning: Empty content on page 1 of document /Users/admin/Projects/MathTestCreator/Documents/further.pdf
  warnings.warn(


In [351]:
# print(material.load_markdown(files[0]))
material.metadata['total_pages']
material.source

'/Users/admin/Projects/MathTestCreator/Documents/further.pdf'

In [174]:
print(len(material.documents.values()))

434


In [375]:
from agents.collection_agent import CollectionAgent
collectionAgent = CollectionAgent()


In [378]:
collectionAgent.create_source_material(files)

aaaaaaaaa
/Users/admin/Projects/MathTestCreator/Documents/further.pdf
already uploaded


In [379]:
collectionAgent.documents

[further.pdf - The document, "Advanced High-School Mathematics" by David B. Surowski, outlines his experiences in developing advanced mathematics notes for students seeking further studies beyond the standard curriculum. It covers various topics including geometry, discrete mathematics, and inferential statistics while emphasizing the importance of logical reasoning in mathematics education.]

In [233]:
from openai import OpenAI
openai = OpenAI()

In [258]:
system_message = """You are a helpful assist. Be polite and courteous. Guide the user through the process of creating a math test.\
                    The user is provided with parameters to set in the interface. Prompt them to use the interface to set the parameters.\
                If they do not want to use parameters you can ask them for more information or make a decision on their behalf.\
                    
    """

In [329]:
collectionAgent = CollectionAgent()
def do_entry(message, history):
        history += [{"role":"user", "content":message}]
        return "", history

def docdata_to_text():
        for i, material in enumerate(collectionAgent.documents):
            yield f"### Document {i+1}: \n\n {repr(material)}"

def chat_gpt(history, parameters={}):
    messages = [{"role": "system", "content": system_message}] + history
    if parameters.values():
        messages.append({"role":"assistant", "content":parameters})
    else:
        messages.append({"role":"assistant", "content":"Do you have any specifications?"})
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply = response.choices[0].message.content
    history += [{"role":"assistant", "content":reply}]
    return history

with gr.Blocks() as ui:

    documents = gr.State()
    collection = gr.State()

    gr.Markdown("##Math Test Creator")
    gr.Markdown("#### Upload files to use as source material for your test")
    with gr.Row():
            with gr.Column():
                file_uploader = gr.File(file_count="multiple")
            with gr.Column():
                gr.Markdown("### Here are the documents you have uploaded")
                doc_data = gr.Markdown(label="Document Data")
    with gr.Row():
        rag_button = gr.Button("Create Your Custom Assistant")
        output = gr.Markdown(label="Response:")
    with gr.Row():
        chatbot = gr.Chatbot(type='messages')
    
    with gr.Row():
        chat_entry = gr.Textbox()
    #Set Parameters
    file_uploader.change(collectionAgent.create_source_material, inputs=[file_uploader], outputs=[documents]).then(docdata_to_text, inputs=[], outputs=[doc_data])

    # file_uploader.change(ragAgent.load_files, inputs=[file_uploader], outputs=[documents]).then(ragAgent.document_data, inputs=[], outputs=[doc_data]).then(ragAgent.setup, inputs=[], outputs=[collection])
    
    chat_entry.submit(do_entry, inputs=[chat_entry, chatbot], outputs=[chat_entry, chatbot]).then(
        chat_gpt, inputs=chatbot, outputs=chatbot)
    


ui.launch()
        

* Running on local URL:  http://127.0.0.1:7948

To create a public link, set `share=True` in `launch()`.


In [ ]:
collectionAgent = CollectionAgent()
def do_entry(message, history):
        history += [{"role":"user", "content":message}]
        return "", history

def docdata_to_text():
        for material in collectionAgent.documents:
            yield f"### Document Successfully Uploaded\n\n {repr(material)}"
def doc_to_df():
     return pd.DataFrame([[doc.name, str(doc.length) + ' ' + doc.length_type] for doc in collectionAgent.documents],columns = ["Name", "Length"])
def set_visibility():
    if customize == "Do you want make custom selections of your material?":
         return gr.update(visible=True)
    
    doc_df_visibility = False if len(collectionAgent.documents) == 0 else True
    return gr.update(visible=doc_df_visibility)
     
def chat_gpt(history, parameters={}):
    messages = [{"role": "system", "content": system_message}] + history
    if parameters.values():
        messages.append({"role":"assistant", "content":parameters})
    else:
        messages.append({"role":"assistant", "content":"Do you have any specifications?"})
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply = response.choices[0].message.content
    history += [{"role":"assistant", "content":reply}]
    return history

with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(type='messages')
    
    with gr.Row():
        chat_entry = gr.Textbox()
    #Set Parameters


    documents = gr.State()
    collection = gr.State()
    gr.Markdown("## Custom Personal Knowledge Assistant")
    interim =  gr.Markdown("#### Upload a File")
    with gr.Row():
            with gr.Column():
                file_uploader = gr.File(file_count="multiple")
            with gr.Column():
                # gr.Markdown("### Here are the documents you have uploaded")
                doc_data = gr.Markdown(label="Document Data")
    with gr.Row():
         doc_df = gr.DataFrame(headers = ["No documents uploaded"]
         )
    with gr.Row():
         customize = gr.Checkbox(["Do you want make custom selections of your material?"],label="Document Selection", )  
    with gr.Row():
         pages = gr.slider(label="Choose your starting page")
    with gr.Row():
        rag_button = gr.Button("Create Your Custom Assistant")
        output = gr.Markdown(label="Response:")
    
    file_uploader.change(collectionAgent.create_source_material, inputs=[file_uploader], outputs=[documents]).then(docdata_to_text, inputs=[], outputs=[doc_data]).then(doc_to_df, outputs=doc_df)

    # file_uploader.change(ragAgent.load_files, inputs=[file_uploader], outputs=[documents]).then(ragAgent.document_data, inputs=[], outputs=[doc_data]).then(ragAgent.setup, inputs=[], outputs=[collection])
    customize.change(show_parameters, inputs=customize, outputs=pages)

    chat_entry.submit(do_entry, inputs=[chat_entry, chatbot], outputs=[chat_entry, chatbot]).then(
        chat_gpt, inputs=chatbot, outputs=chatbot)
    


ui.launch()
        

* Running on local URL:  http://127.0.0.1:7876

To create a public link, set `share=True` in `launch()`.


In [ ]:
ui.close()

In [1]:
import gradio as gr
import pandas as pd

# Initial parameters for the editable table (as a list of lists)
initial_data = [
    ["Learning Rate", 0.001],
    ["Batch Size", 32],
    ["Epochs", 10],
]

# Function to process and display updated parameters
def update_parameters(dataframe):
    # Convert the editable dataframe into a Pandas DataFrame
    df = pd.DataFrame(dataframe, columns=["Parameter", "Value"])
    # Process or validate the data if needed
    return f"Updated Parameters:\n{df.to_string(index=False)}"

# Gradio Interface
with gr.Blocks() as demo:
    gr.Markdown("## Editable Table of Parameters")
    
    # Editable dataframe
    parameter_table = gr.Dataframe(
        value=initial_data,
        headers=["Parameter", "Value"],
        datatype=["str", "number"],  # Specify data types for columns
        row_count=(3, "dynamic"),  # Allow dynamic row addition
        col_count=(2, "fixed"),  # Fixed number of columns
        interactive=True,  # Make the table editable
        label="Edit Parameters"
    )
    
    # Output area to display the processed parameters
    output = gr.Textbox(label="Output", interactive=False, lines=5)
    
    # Button to submit changes
    update_button = gr.Button("Update Parameters")
    
    # Update button triggers the update process
    update_button.click(update_parameters, inputs=[parameter_table], outputs=[output])

# Launch the Gradio app
demo.launch(server_port=7860)
demo.close()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Closing server running on port: 7860


In [389]:
demo.close()